In [1]:
pip install transformers torch datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [5]:
from datasets import Dataset, DatasetDict

data = {
    "train": Dataset.from_dict({
        "context": [
            "Mevlana, 13. yüzyıl Pers şairi, yargıç ve İslam alimi.",
            "Mevlana Celaleddin-i Rumi, Sufizmin en önemli temsilcilerinden biridir.",
            "Şems ile Mevlana'nın dostluğu, Mevlana'nın şiirlerinde derin bir iz bırakmıştır."
        ],
        "question": [
            "Mevlana hangi yüzyılda yaşamıştır?",
            "Mevlana'nın tam adı nedir?",
            "Mevlana'nın hayatında Şems'in rolü nedir?"
        ],
        "answers": [
            {"text": ["13. yüzyıl"], "answer_start": [9]},
            {"text": ["Mevlana Celaleddin-i Rumi"], "answer_start": [0]},
            {"text": ["dostluğu"], "answer_start": [15]}
        ]
    }),
    "test": Dataset.from_dict({
        "context": [
            "Mevlana, Mevlevilik tarikatının kurucusudur."
        ],
        "question": [
            "Mevlana hangi tarikatın kurucusudur?"
        ],
        "answers": [
            {"text": ["Mevlevilik"], "answer_start": [9]}
        ]
    })
}


dataset = DatasetDict(data)


In [6]:
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = RobertaForQuestionAnswering.from_pretrained('roberta-base')


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def prepare_features(examples):

    tokenized_inputs = tokenizer(
        examples['question'], examples['context'],
        truncation=True, padding="max_length", max_length=512,
        return_offsets_mapping=True
    )
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):

        answer = examples['answers'][i]
        start_char = answer['answer_start'][0]
        end_char = start_char + len(answer['text'][0])


        start_position = next((j for j, offset in enumerate(offsets) if offset[0] <= start_char < offset[1]), None)
        end_position = next((j for j, offset in enumerate(offsets) if offset[0] < end_char <= offset[1]), None)


        if start_position is None or end_position is None:
            start_position = 0
            end_position = 0

        start_positions.append(start_position)
        end_positions.append(end_position)

    tokenized_inputs.update({
        'start_positions': start_positions,
        'end_positions': end_positions
    })
    return tokenized_inputs


dataset = dataset.map(prepare_features, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [12]:
pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 1.1 MB/s eta 0:00:00


In [1]:
pip install transformers[torch] -U


In [8]:
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering


tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = RobertaForQuestionAnswering.from_pretrained('roberta-base')


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test']
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,6.171900
2,No log,6.170410
3,No log,6.167393


TrainOutput(global_step=3, training_loss=6.240360260009766, metrics={'train_runtime': 78.0721, 'train_samples_per_second': 0.115, 'train_steps_per_second': 0.038, 'total_flos': 2351670810624.0, 'train_loss': 6.240360260009766, 'epoch': 3.0})

In [15]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

def ask_question(context, question):
    result = qa_pipeline({
        'context': context,
        'question': question
    })
    return result['answer']


context = "Mevlana, on üçüncü yüzyıl Pers şairi, yargıç ve İslam alimi. Mevlana Celaleddin-i Rumi, Sufizmin en önemli temsilcilerinden biridir."
question = "Mevlana kimdir?"


answer = ask_question(context, question)
print(f"Soru: {question}")
print(f"Cevap: {answer}")


Soru: Mevlana kimdir?
Cevap: Mevlana Celaleddin-i Rumi,
